In [1]:
# Imports
from pprint import pprint
import json
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians

In [6]:
gps_data = []
with open('gps_dump.json') as file:
    for line in file:
        parts = line.split(' ',1) 
        if '/gps/rmc' in parts[0]:
            gps_data.append(parts)
pprint(gps_data[:5])

[['owntracks/fartyg/alv-snabben-5/gps/rmc',
  '$GPRMC,100312.00,A,5742.37365,N,01157.35833,E,3.552,272.92,110519,,,A*63\n'],
 ['owntracks/tag/9035/gps/rmc',
  '$GPRMC,100312.00,A,5810.91179,N,01333.55914,E,0.966,,110519,,,A*70\n'],
 ['owntracks/tag/3142/gps/rmc',
  '$GPRMC,100312.00,A,5804.35956,N,01149.48552,E,0.016,,110519,,,A*7D\n'],
 ['owntracks/tag/9084/gps/rmc',
  '$GPRMC,100312.00,A,5821.24739,N,01155.42373,E,0.205,279.80,110519,,,A*65\n'],
 ['owntracks/sparvagn/438/gps/rmc',
  '$GPRMC,100312.00,A,5742.42229,N,01159.71184,E,0.007,,110519,,,A*77\n']]


In [7]:
data_gps = []
import pynmea2
for item in gps_data:
    data_list = item[1].split(',')
    msg = pynmea2.parse(item[1])
    try:
        data_gps.append((item[0].replace('owntracks/','').replace('/gps/rmc',''),
                        data_list[9],
                        msg.timestamp,
                        data_list[2],
                        msg.latitude,
                        msg.longitude))
    except:
        pass
df_gps = pd.DataFrame(data=data_gps, columns=['vehicle','date','time','active or void','lat','lon'])
display(df_gps.head())
# df_gps.to_csv(path_or_buf='gps_data.csv',index=False)

,vehicle,date,time,active or void,lat,lon
0,fartyg/alv-snabben-5,110519,10:03:12,A,57.706227,11.955972
1,tag/9035,110519,10:03:12,A,58.181863,13.559319
2,tag/3142,110519,10:03:12,A,58.072659,11.824759
3,tag/9084,110519,10:03:12,A,58.354123,11.923729
4,sparvagn/438,110519,10:03:12,A,57.707038,11.995197


In [4]:
trams_with_connection_loss = df_gps[df_gps['active or void'] == 'V']
# display(trams_with_connection_loss)
trams = df_gps.groupby('vehicle')
dict_trams = {}

for name,group in trams:
    tram_locations = []
    
    conn = None
    pre_loc = None
    for idx, row in group.iterrows():
#         print(row)
        if conn is None:
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'A' and row['active or void'] == 'V':
            tram_locations.append(pre_loc)
            conn = row['active or void']
        elif conn == row['active or void'] and conn == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'V' and row['active or void'] == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
            tram_locations.append(pre_loc)
        else:
            pass
    if  tram_locations:
        dict_trams[name] = tram_locations
        
# pprint(dict_trams)
# with open('tram_lost_locations.json', 'w') as json_file:  
#     json.dump(dict_trams, json_file)


In [8]:
# locations where connection where lost and where regained
tram_locations = []
for name,group in trams:
    conn = None
    pre_loc = None
    for idx, row in group.iterrows():
#         print(row)
        if conn is None:
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'A' and row['active or void'] == 'V':
            pre_loc = (0,) + pre_loc
            tram_locations.append(pre_loc)
            conn = row['active or void']
        elif conn == row['active or void'] and conn == 'A':
            conn = row['active or void']
            pre_loc = (row['lat'], row['lon'])
        elif conn == 'V' and row['active or void'] == 'A':
            conn = row['active or void']
            pre_loc = (1, row['lat'], row['lon'])
            tram_locations.append(pre_loc)
        else:
            pass
pprint(tram_locations[:5])
# with open('tram_lost_locations.csv', 'w') as f:  
#     for item in tram_locations:
#         f.write(str(item[0])+','+str(item[1])+','+str(item[2])+'\n')

[(0, 57.692002333333335, 11.990103666666666),
 (1, 57.688943333333334, 11.969925666666667),
 (0, 57.6923, 11.9899745),
 (1, 57.6904425, 11.9747785),
 (0, 57.6512395, 11.907624666666667)]


In [11]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # calculate distance between two lat,lon locations
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
#     print(distance)
    return distance


active_trams = df_gps[df_gps['active or void'] == 'A']
active_trams.sort_values(by=['time','lat','lon']).head(20)


,vehicle,date,time,active or void,lat,lon
40769,tag/9078,110519,10:00:57,A,58.327802,12.261468
40974,tag/9078,110519,10:00:58,A,58.327442,12.261453
40975,tag/9078,110519,10:00:59,A,58.327079,12.261441
40976,tag/9078,110519,10:01:00,A,58.326715,12.261429
40977,tag/9078,110519,10:01:01,A,58.326349,12.261418
40978,tag/9078,110519,10:01:02,A,58.325983,12.261405
40979,tag/9078,110519,10:01:03,A,58.325614,12.261394
40980,tag/9078,110519,10:01:04,A,58.325241,12.261381
40981,tag/9078,110519,10:01:05,A,58.324871,12.261373
40982,tag/9078,110519,10:01:06,A,58.324516,12.261367


In [ ]:
# calculate the distance between data points - very costly...
active_trams.set_index('time', inplace=True)
active_trams = active_trams.drop(columns=['date','active or void'])

sLength = len(active_trams['vehicle'])
active_trams['distance to prior']= np.zeros(sLength)

pre_row = None
for idx, row in active_trams.iterrows():
    if pre_row is None:
        pre_row = row
    else:
        distance = calculate_distance(pre_row['lat'], pre_row['lon'], row['lat'], row['lon'])
        active_trams.set_value(idx,'distance to prior',distance)
        pre_row = row
display(active_trams)

C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [7]:
# prepare vehicle, lat and lon for demo of queuing algorithm
new_locs = df_gps.drop(columns=['date','time','active or void'])
new_locs.to_csv(path_or_buf='new_locations.csv',index=False)

In [5]:
print(len(df_gps['vehicle'].unique()))

288
